Import libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

get the page

In [2]:
booksHTML = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever").text

In [3]:
soup = BeautifulSoup(booksHTML, 'lxml')

In [4]:
booksDiv = soup.find('div', id = "all_votes")

In [5]:
booksCards = booksDiv.find_all('tr')

In [6]:
print(len(booksCards))

100


get titles

In [7]:
titles = [booksCards[i].find('a', class_ = "bookTitle").span.text for i in range(len(booksCards))]

In [ ]:
for j in titles:
    print(j)

get authors

In [9]:
authors = [booksCards[i].find('div', class_ = "authorName__container").span.text for i in range(len(booksCards))]

In [ ]:
for j in authors:
    print(j)

In [11]:
import re

get score

In [12]:
scores = [int(re.sub(",", "", booksCards[i].find('div', style = "margin-top: 5px").a.text[7:])) for i in range(len(booksCards))]

In [ ]:
for k in scores:
    print(k)

get votes

In [14]:
regex = r"\d{1,}\W\w\d{1,}\speople"
votes = [re.findall(regex, booksCards[i].find('div', style = "margin-top: 5px").span.text[:]) for i in range(len(booksCards))]

In [ ]:
print(votes)

In [16]:
votes_striped = [int(re.sub("[,people\s]", "", i[0])) for i in votes]

In [ ]:
print(votes_striped)

get average rating

In [18]:
rating = [booksCards[i].find('span', class_ = "greyText smallText uitext").span.text[1:5] for i in range(len(booksCards))]

In [ ]:
# removing NULL ratings
i = rating.index('eall')
rating2 = rating[:i] + ["0.00"] + rating[i+1:]
print(rating2)

In [20]:
print(len(rating2))

100


In [21]:
# converting string to numeric
avgRating = [float(i) for i in rating2]

create DataFrame

In [22]:
# creating the DataFrame
books_df = pd.DataFrame(list(zip(titles, authors, scores, votes_striped, avgRating)), columns = ["Title", "Author", "Score", "Votes", "Average Rating"])

In [23]:
# one book is repaeted 
# to drop it permanently 
books_df.drop_duplicates(subset = "Title",inplace = True)
# setting subset = Title cuz it will find and remove duplicates of the column Title

write to csv file

In [24]:
# writing the Data to csv file
books_df.to_csv('Books-DataFrame.csv')

ploting with plotly

In [25]:
import plotly.express as px

In [26]:
# histogram of scores of books
fig = px.histogram(books_df["Score"], template = "plotly_dark", nbins=200)
fig.show()

In [27]:
# in bar chart
# we need to pass x-axis and y-axis
# x is catergorical column and y is numerical column
fig = px.bar(x = books_df["Title"], y = books_df["Score"], template = 'plotly_dark', labels = {'x': 'Title', 'y':'Score'})
fig.show()

In [28]:
# bar chart against title and rating
fig = px.bar(x = books_df["Title"], y = books_df["Average Rating"], template = "plotly_dark", labels={"x": "Title", "y":"Average Rating"})
fig.show()

In [29]:
# similarly line chart for same x and same y
fig = px.line(x = books_df["Title"], y = books_df["Score"], template = "plotly_dark")
fig.show()

In [30]:
# scatter plots can be used for two numerical columns
fig = px.scatter(data_frame=books_df, x = "Score", y = "Votes", template = 'plotly_dark')
fig.show()

In [31]:
# scatter plot across score and average rating
fig = px.scatter(data_frame = books_df,
                 x = "Score",
                 y = "Average Rating",
                 template = "plotly_dark",
                 title = "Average Rating by Score")
fig.show()